In [1]:
# # Ensure pip, wheel, and setuptools are up to date
# !pip install --upgrade pip setuptools wheel

# # Install dependencies via pip directly, including those available on PyPI
# !pip install autograd pandas plotly scikit-learn scipy numpy wandb ray torch torchvision torchaudio botorch Cython pyyaml
# !git clone https://github.com/Queimo/DGEMO
# %cd DGEMO
# # pymoo specific installation from a certain commit
# !rm -rf pymoo
# !git clone --no-checkout https://github.com/anyoptimization/pymoo.git
# %cd pymoo
# !git checkout d8af6a4
# !python setup.py build_ext --inplace --cythonize
# !pip install .
# %cd ..

# # Install pygco - note that this might require a custom installation if it's not directly available via pip
# !pip install pygco openpyxl

# !pip install numpy --force-reinstall
# !pip install cma --force-reinstall

In [2]:

# os.environ["OMP_NUM_THREADS"] = "1"  # speed up
import numpy as np
from problems.common import build_problem
from mobo.algorithms import get_algorithm
from visualization.data_export import DataExport
from utils import save_args
from ref_point import RefPoint
import torch
import gc
from mobo.mobo import MOBO

import numpy as np
from mobo.surrogate_problem import SurrogateProblem
from mobo.utils import (
    Timer,
    find_pareto_front,
    calc_hypervolume,
)


c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# algo_name = 'raqlognehvidet'
algo_name = 'marsdet'
# algo_name = 'mars'
ALGO = get_algorithm(algo_name)


In [4]:

class MOBOEXP(ALGO):
    """
    Base class of algorithm framework, inherit this class with different configs to create new algorithm classes
    """

    def __init__(self, problem, n_iter, ref_point_handler, framework_args, batch_size=6):
        
        self.df = problem.df_mean_std
        self.batch_size = batch_size
        
        super().__init__(problem, n_iter, ref_point_handler, framework_args)

    def step(self):
        
        timer = Timer()

        # data normalization
        self.transformation.fit(self.X, self.Y)
        X, Y, rho = self.transformation.do(self.X, self.Y, self.rho)

        # X, Y, rho = self.transformation.do(self.X, self.Y, self.rho)

        # # build surrogate models
        self.surrogate_model.fit(X, Y, rho)
        timer.log("Surrogate model fitted")

        # define acquisition functions
        self.acquisition.fit(X, Y)

        # solve surrogate problem
        surr_problem = SurrogateProblem(
            self.real_problem,
            self.surrogate_model,
            self.acquisition,
            self.transformation,
        )
        if self.sample_num >= len(self.df):

            solution = self.solver.solve(surr_problem, X, Y, rho)
            timer.log("Surrogate problem solved")
            # batch point selection
            self.selection.fit(X, Y)
            X_next, self.info = self.selection.select(
                solution, self.surrogate_model, self.status, self.transformation
            )
            timer.log("Next sample batch selected")

            # update dataset
            # Y_next, rho_next = self.real_problem.evaluate(X_next, return_values_of=['F', 'rho'])
            # Don't need to evaluate real problem because we don't have the data yet
            # evaluate prediction of X_next on surrogate model
            val = self.surrogate_model.evaluate(self.transformation.do(x=X_next), std=True, noise=True)
            Y_next_pred_mean, rho_next_pred = self.transformation.undo(y=val['F'], rho=val['rho_F'])
            rho_next = rho_next_pred
            Y_next = Y_next_pred_mean
            Y_next_pred_std = val['S']
            acquisition, _, _ = self.acquisition.evaluate(val)
        
        else:
            
            # only solve for current pareto front with NSGA of gp
            solution = self.solver.nsga2_solve(surr_problem, X, Y)
            self.solver.solution = solution
            timer.log("Surrogate problem solved")
            X_next = self.df.iloc[self.sample_num:self.sample_num+self.batch_size][self.problem.var_cols].values
            Y_next = -1 * self.df.iloc[self.sample_num:self.sample_num+self.batch_size][self.problem.obj_cols].values # we assume minimzation
            rho_next = self.df.iloc[self.sample_num:self.sample_num+self.batch_size][[col.replace("_mean","_std") for col in self.problem.obj_cols]].values
            
            val = self.surrogate_model.evaluate(self.transformation.do(x=X_next), std=True, noise=True)
            Y_next_pred_mean, rho_next_pred = self.transformation.undo(y=val['F'], rho=val['rho_F'])
            Y_next_pred_std = val['S']
            acquisition, _, _ = self.acquisition.evaluate(val)

        
        if self.real_problem.n_constr > 0:
            Y_next = Y_next[0]
            rho_next = rho_next[0]
        self._update_status(X_next, Y_next, rho=rho_next)
        timer.log("New samples evaluated")

        # statistics
        self.global_timer.log("Total runtime", reset=False)
        print(
            "Total evaluations: %d, hypervolume: %.4f\n"
            % (self.sample_num, self.status["hv"])
        )

        # return new data iteration by iteration
        return X_next, Y_next, rho_next, Y_next_pred_mean, Y_next_pred_std, acquisition



In [5]:

def run_experiment(args, framework_args):
    # load arguments

    # set seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    # build problem, get initial samples
    problem, true_pfront, X_init, Y_init, rho_init = build_problem(
        args.problem, args.n_var, args.n_obj, args.n_init_sample, args.n_process
    )
    args.n_var, args.n_obj = problem.n_var, problem.n_obj

    ref_point_handler = RefPoint(
        args.problem, args.n_var, args.n_obj, n_init_sample=args.n_init_sample
    )

    args.ref_point = ref_point_handler.get_ref_point(is_botorch=False)

    # initialize optimizer
    
    optimizer = MOBOEXP(
        problem, args.n_iter, ref_point_handler, framework_args, batch_size=args.batch_size
    )
    
    

    # save arguments & setup logger
    save_args(args, framework_args)
    print(problem, optimizer, sep="\n")

    # initialize data exporter
    exporter = DataExport(optimizer, X_init, Y_init, rho_init, args)

    # optimization
    solution = optimizer.solve(X_init, Y_init, rho_init)

    # export true Pareto front to csv
    if true_pfront is not None:
        exporter.write_truefront_csv(true_pfront)

    for step in range(args.n_iter):
        # get new design samples and corresponding performance
        X_next, Y_next, rho_next, Y_next_pred_mean, Y_next_pred_std, acq = next(
            solution
        )
        # update & export current status to csv
        exporter.update(X_next, Y_next, Y_next_pred_mean, Y_next_pred_std, acq, rho_next)

        gc.collect()

        exporter.write_csvs()
        exporter.save_psmodel()
    
    


In [8]:

from arguments import get_args

args, framework_args = get_args()

args.algo = algo_name
args.problem = 'exp4d'
args.n_iter = 1
args.n_init_sample = 12
args.batch_size = 6
args.subfolder = 'optimization_2'
framework_args["solver"]["batch_size"] = args.batch_size
framework_args["selection"]["batch_size"] = args.batch_size

run_experiment(args, framework_args)


    C_ZnCl_mean  C_NaOH/C_ZnCl_mean  C_NaOH_mean  Aspect Ratio_mean  \
id                                                                    
0          0.28                1.76       0.4928           3.473333   
1          0.66                2.59       1.7094           2.293333   
2          0.86                0.80       0.6880           0.760000   
3          0.51                2.75       1.4025           2.373333   
4          0.35                1.22       0.4270           1.163333   
5          0.93                3.17       2.9481           2.280000   
6          0.70                1.44       1.0080           1.183333   
7          0.10                2.27       0.2270           3.930000   
8          0.18                0.65       0.1170           4.190000   
9          0.77                2.99       2.3023           2.360000   
10         0.98                2.00       1.9600           2.510000   
11         0.39                2.45       0.9555           2.683333   

    P